In [29]:
# import logging

# from autogen_core import EVENT_LOGGER_NAME

# logging.basicConfig(level=logging.WARNING)
# logger = logging.getLogger(EVENT_LOGGER_NAME)
# logger.addHandler(logging.StreamHandler())
# logger.setLevel(logging.INFO)

In [30]:
from dotenv import load_dotenv
load_dotenv()

True

In [31]:
import asyncio

from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_core.models import ModelInfo, UserMessage

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import StructuredMessage
from autogen_agentchat.ui import Console
from autogen_agentchat.base import TaskResult

from autogen_agentchat.conditions import ExternalTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_core import CancellationToken



In [32]:
from model_config import OpenAIConfig
def _create_model_client() -> OpenAIChatCompletionClient:
    """Create the model client used by all agents."""
    config = OpenAIConfig().from_env()
    return OpenAIChatCompletionClient(
        model=config.model,
        base_url=config.base_url,
        api_key=config.api_key,
        model_info=config.model_info,
        temperature=0.15,
    )

In [33]:
model_client = _create_model_client()

In [34]:
assistant_system_message = ('You are an expert Python developer with deep knowledge of software testing, including unit tests, integration tests, and best practices for test-driven development (TDD). Your task is to analyze Python code and generate high-quality, maintainable, and efficient test cases. Instructions:  1. Understand the Code:    - Carefully review the provided Python code, including functions, classes, and dependencies.    - Identify edge cases, expected behavior, and potential failure points.  2. Choose a Testing Framework:     - Default to `pytest` if no framework is specified.     - Ensure compatibility with the code’s dependencies (e.g., `mock` for external services).  3. Generate Tests:  - Write tests that cover:       - Happy Path (normal operation).       - Edge Cases (empty inputs, invalid types, boundary conditions).       - Error Handling (exceptions, invalid states).     - Include descriptive docstrings or comments explaining each test’s purpose.     - Use fixtures (`pytest`) or `setUp/tearDown` (`unittest`) where needed.  4. Output Format:     - Return the test code in a complete, executable format.     - If the original code has bugs, note them and write tests to catch them.  When done, say TERMINATE.')

In [35]:
primary_agent = AssistantAgent(
    "test_writer",
    model_client=model_client,
    system_message=assistant_system_message,
)

In [36]:
critic_system_message = "You are a task verification assistant who is working with a test writer agent to solve tasks. At each point, check if the task has been completed as requested by the user. You MUST run and validate all the tests provided by test_writer as completed. If the test_writer responds and the task has not yet been completed, respond with what is left to do and then say 'keep going' (provide constructive feedback). If and only when the task has been completed, summarize and present a final answer that directly addresses the user task in detail and then respond with APPROVE."
# critic_system_message = "Provide constructive feedback. Respond with 'APPROVE' to obly after ALL your feedbacks are addressed by test_writer."

In [37]:
critic_agent = AssistantAgent(
    "critic",
    model_client=model_client,
    system_message=critic_system_message
)

In [38]:
text_termination = TextMentionTermination("APPROVE")
team = RoundRobinGroupChat([primary_agent, critic_agent], termination_condition=text_termination)

In [39]:
await team.reset()  # Reset the team for a new task.
await Console(team.run_stream(task="Write a function in Python and several tests for it."))

---------- TextMessage (user) ----------
Write a function in Python and several tests for it.
---------- TextMessage (test_writer) ----------
Sure, let's start by writing a simple Python function and then we'll create tests for it. Here's a function that calculates the factorial of a number:

```python
def factorial(n):
    if n < 0:
        raise ValueError("Factorial is not defined for negative numbers")
    elif n == 0:
        return 1
    else:
        result = 1
        for i in range(1, n + 1):
            result *= i
        return result
```

Now, let's write tests for this function using `pytest`. We'll cover the happy path, edge cases, and error handling.

```python
import pytest

def test_factorial_happy_path():
    assert factorial(5) == 120
    assert factorial(10) == 3628800
    assert factorial(0) == 1

def test_factorial_edge_cases():
    assert factorial(1) == 1
    assert factorial(2) == 2

def test_factorial_error_handling():
    with pytest.raises(ValueError):
    

/Users/ivangorbunov/anaconda3/lib/python3.11/site-packages/autogen_agentchat/agents/_assistant_agent.py:1107: UserWarning: Resolved model mismatch: openai/Devstral != Devstral. Model mapping in autogen_ext.models.openai may be incorrect. Set the model to Devstral to enhance token/cost estimation and suppress this warning.
  model_result = await model_client.create(


---------- TextMessage (critic) ----------
The task has not yet been completed. The test writer has provided a function and some tests, but the tests have not been run or validated. Here's what's left to do:

1. Run the tests to ensure they pass.
2. Validate the results to confirm the function works as expected.

Keep going.
---------- TextMessage (test_writer) ----------
I apologize for the misunderstanding. Since I can't directly run the tests, I'll provide you with a complete, executable format of the test code using `pytest`. You can run these tests in your local environment to ensure they pass and validate the results.

Here's the complete test code:

```python
import pytest

def factorial(n):
    if n < 0:
        raise ValueError("Factorial is not defined for negative numbers")
    elif n == 0:
        return 1
    else:
        result = 1
        for i in range(1, n + 1):
            result *= i
        return result

def test_factorial_happy_path():
    assert factorial(5) == 

Error processing publish message for test_writer_42ea8b7a-47c9-4847-b342-5a146badfbec/42ea8b7a-47c9-4847-b342-5a146badfbec
Traceback (most recent call last):
  File "/Users/ivangorbunov/anaconda3/lib/python3.11/site-packages/httpx/_transports/default.py", line 72, in map_httpcore_exceptions
    yield
  File "/Users/ivangorbunov/anaconda3/lib/python3.11/site-packages/httpx/_transports/default.py", line 377, in handle_async_request
    resp = await self._pool.handle_async_request(req)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ivangorbunov/anaconda3/lib/python3.11/site-packages/httpcore/_async/connection_pool.py", line 256, in handle_async_request
    raise exc from None
  File "/Users/ivangorbunov/anaconda3/lib/python3.11/site-packages/httpcore/_async/connection_pool.py", line 236, in handle_async_request
    response = await connection.handle_async_request(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ivangorbunov/anaconda3/lib/python3.

CancelledError: 

In [7]:
# result = await team.run(task="Write a simple but useful function in Python and a test for it.")
# print(result.messages[1].content)

In [10]:
# pip install --upgrade autogen-agentchat

In [9]:
import asyncio
from autogen_agentchat.agents import CodeExecutorAgent, ApprovalRequest, ApprovalResponse
from autogen_agentchat.messages import TextMessage
from autogen_ext.code_executors.docker import DockerCommandLineCodeExecutor
from autogen_core import CancellationToken


def simple_approval_func(request: ApprovalRequest) -> ApprovalResponse:
    """Simple approval function that requests user input for code execution approval."""
    print("Code execution approval requested:")
    print("=" * 50)
    print(request.code)
    print("=" * 50)

    while True:
        user_input = input("Do you want to execute this code? (y/n): ").strip().lower()
        if user_input in ['y', 'yes']:
            return ApprovalResponse(approved=True, reason='Approved by user')
        elif user_input in ['n', 'no']:
            return ApprovalResponse(approved=False, reason='Denied by user')
        else:
            print("Please enter 'y' for yes or 'n' for no.")


async def run_code_executor_agent() -> None:
    # Create a code executor agent that uses a Docker container to execute code.
    code_executor = DockerCommandLineCodeExecutor(work_dir="coding")
    await code_executor.start()
    code_executor_agent = CodeExecutorAgent(
        "code_executor",
        code_executor=code_executor,
        approval_func=simple_approval_func
    )

    # Run the agent with a given code snippet.
    task = TextMessage(
        content='''Here is some code
```python
print('Hello world')
```
''',
        source="user",
    )
    response = await code_executor_agent.on_messages([task], CancellationToken())
    print(response.chat_message)

    # Stop the code executor.
    await code_executor.stop()


await run_code_executor_agent()
# asyncio.run(run_code_executor_agent())

Code execution approval requested:
```python
print('Hello world')

```


Do you want to execute this code? (y/n):  y


id='76fcaa10-f6a1-44c3-9bd4-1a5dcd132e51' source='code_executor' models_usage=None metadata={} created_at=datetime.datetime(2025, 8, 12, 18, 58, 32, 703625, tzinfo=datetime.timezone.utc) content='Hello world\n' type='TextMessage'


In [1]:
# !pip install "autogen-ext[docker]"
# !pip install docker

In [ ]:
# CodeExecutorAgent
# sources (Sequence[str], optional) – Check only messages from the specified agents for the code to execute
# supported_languages

# The code executor only processes code that is properly formatted in markdown code blocks using triple backticks

In [13]:
# pip install "autogen-ext[file-surfer]"